In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from skimage.feature import hog

In [ ]:
# FOR GOOGLE COLAB
# import pickle
# import os
# from urllib.request import urlretrieve

# def download(url, file):
#     """
#     Download file from <url>
#     :param url: URL to file
#     :param file: Local file path
#     """
#     if not os.path.isfile(file):
#         print('Downloading ' + file + '...')
#         urlretrieve(url, file)
#         print('Download Finished')

# download('https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip', 'vehicles.zip')
# download('https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip', 'non-vehicles.zip')

# import zipfile
# zip_ref = zipfile.ZipFile('vehicles.zip', 'r')
# zip_ref.extractall(".")
# zip_ref.close()
# zip_ref = zipfile.ZipFile('non-vehicles.zip', 'r')
# zip_ref.extractall(".")
# zip_ref.close()

# import glob
# veh_images = glob.glob('./vehicles/**/*.png' , recursive=True)
# cars = []
# notcars = []

# for image in veh_images:
#         cars.append(image)
        
# # print(len(cars), "is number of car photos") 
# nonveh_images = glob.glob('./non-vehicles/**/*.png' , recursive=True)
# for image in nonveh_images:
#         notcars.append(image)

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    draw_img = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    return draw_img
# bboxes = [((100, 100), (200, 200))]

In [ ]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    ghist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    bhist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return rhist, ghist, bhist, bin_centers, hist_features

In [ ]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=True, 
                     feature_vec=True):
                         
    # TODO: Complete the function body and returns
    hog_features, hog_image = hog(img, orientations=orient,
                          pixels_per_cell=(pix_per_cell, pix_per_cell), 
                          cells_per_block=(cell_per_block, cell_per_block), 
                          visualise=vis, feature_vector=feature_vec,
                          block_norm="L2-Hys")
    if vis==True:
        return hog_features, hog_image
    else:
        return hog_features

In [ ]:
def bin_spatial(img, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return features

In [ ]:
import glob
veh_images = glob.glob('../vehicles/**/*.png' , recursive=True)
cars = []
notcars = []

for image in veh_images:
        cars.append(image)
        
# print(len(cars), "is number of car photos") 
nonveh_images = glob.glob('../non-vehicles/**/*.png' , recursive=True)
for image in nonveh_images:
        notcars.append(image)
        
# print(len(notcars) , "is number of non vehicle photos")

# Data Exploration

In [ ]:
# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    # Define a key "image_shape" and store the test image shape 3-tuple
    x = mpimg.imread(car_list[0])
    data_dict["image_shape"] = np.shape(x)
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = x.dtype
    # Return data_dict
    return data_dict
    
data_info = data_look(cars, notcars)

print('Your function returned a count of', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])
# Just for fun choose random car / not-car indices and plot example images   
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))
    
# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])


# Plot the examples
fig = plt.figure()
plt.subplot(121)
plt.imshow(car_image)
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(notcar_image)
plt.title('Example Not-car Image')

In [ ]:
# Generate a random index to look at a car image
ind = np.random.randint(0, len(cars))
# Read in the image
image = mpimg.imread(cars[ind])
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Call our function with vis=True to see an image output
features, hog_image = get_hog_features(gray, orient= 9, 
                        pix_per_cell= 8, cell_per_block= 2, 
                        vis=True, feature_vec=False)


# Plot the examples
fig = plt.figure()
plt.subplot(121)
plt.imshow(image, cmap='gray')
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(hog_image, cmap='gray')
plt.title('HOG Visualization')